# 0.5 Generate Noisy Data and Fitting

This notebook is used to generate and fit all the data required for the paper. We will generate data for the following noise cases: 

1, 2, 3, 4, 5, 6, 7 STD

## Imports

In [1]:
import sys

sys.path.append("../../")
# sys.path.append("/home/ferroelectric/Documents/m3_learning/m3_learning/src")
sys.path.append('../../src')

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from m3_learning.be.dataset import BE_Dataset
from m3_learning.viz.printing import printer
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model
from m3_learning.util.file_IO import download_and_unzip


2024-02-10 21:15:49.626550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 21:15:50.760819: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/cv2/../../lib64:
2024-02-10 21:15:50.760905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/

## Loading data for SHO fitting


In [3]:
# Download the data file from Zenodo
url = 'https://zenodo.org/record/7774788/files/PZT_2080_raw_data.h5?download=1'

# Specify the filename and the path to save the file
filename = '/data_raw_unmod.h5'
save_path = './Data'

# download the file
download_and_unzip(filename, url, save_path)

Using files already downloaded


In [4]:
data_path = save_path + '/' + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

No spectroscopic datasets found as attributes of /Measurement_000/Channel_000/Position_Indices
No position datasets found as attributes of /Raw_Data-SHO_Fit_000/Spectroscopic_Values


/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/m3_learning/util/preprocessing.py:43: RuntimeWarning: invalid value encountered in divide
  return (data - self.mean)/self.std


/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Noisy_Data_2
    ├ Noisy_Data_3
    ├ Noisy_Data_4
    ├ Noisy_Data_5
    ├ Noisy_Data_6
    ├ Noisy_Data_7
    ├ Noisy_Data_8
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
├ Noisy_Data_1_SHO_Fit
  --------

## Generates Noisy Data

This function will generate noisy records and save them as an h5_main file in the USID format. This allows the data to be computed with the Pycroscopy SHO Fitter. 

In [5]:
# calculates the standard deviation and uses that for the noise
noise_STD = np.std(dataset.get_original_data)

# prints the standard deviation
print(noise_STD)

0.0038833667


In [6]:
dataset.generate_noisy_data_records(noise_levels = np.arange(1,9), 
                                    verbose=True, 
                                    noise_STD=noise_STD)

The STD of the data is: 0.0038833667058497667
Adding noise level 1
Adding noise level 2
Adding noise level 3
Adding noise level 4
Adding noise level 5
Adding noise level 6
Adding noise level 7
Adding noise level 8


## SHO fits on all the datasets

This will take some time, Each fit takes about 10 minutes to complete. 

In [7]:
out = [f"Noisy_Data_{i}" for i in np.arange(1,9)]
out.append("Raw_Data")

for data in out:
    print(f"Fitting {data}")
    dataset.SHO_Fitter(dataset = data, h5_sho_targ_grp = f"{data}_SHO_Fit", max_mem=1024*64, max_cores= 20)

Fitting Noisy_Data_1
Working on:
./Data//data_raw_unmod.h5
['Y', 'X'] [60, 60]


SHO Fits will be written to:
./Data/data_raw_unmod.h5


Consider calling test() to check results before calling compute() which computes on the entire dataset and writes results to the HDF5 file
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 finished parallel computation
Rank 0 - 7% complete. Time remaining: 2.02 mins
Rank 0 finished parallel computation
Rank 0 - 15% complete. Time remaining: 2.47 mins
Rank 0 finished parallel computation
Rank 0 - 23% complete. Time remaining: 2.06 mins
Rank 0 finished parallel computation
Rank 0 - 31% complete. Time remaining: 1.77 mins
Rank 0 finished parallel computation
Rank 0 - 39% complete. Time remaining: 1.52 

/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Resuming computation. 0% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 - 6% complete. Time remaining: 3.35 mins
Rank 0 - 13% complete. Time remaining: 2.56 mins
Rank 0 - 20% complete. Time remaining: 2.24 mins
Rank 0 - 26% complete. Time remaining: 2.0 mins
Rank 0 - 33% complete. Time remaining: 1.77 mins
Rank 0 - 40% complete. Time remaining: 1.47 mins
Rank 0 - 46% complete. Time remaining: 1.32 mins
Rank 0 - 53% complete. Time remaining: 1.15 mins
Rank 0 - 60% complete. Time remaining: 59.05 sec
Rank 0 - 66% complete. Time remaining: 47.88 sec
Rank 0 - 73% complete. Time remaining: 37.78 sec
Rank 0 - 80% complete. Time remaining: 27.26 sec
Rank 0 - 87% complete. Time remaining: 17.4 sec
Rank 0 - 93% complete. 

/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Resuming computation. 0% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 - 6% complete. Time remaining: 2.14 mins
Rank 0 - 13% complete. Time remaining: 2.06 mins
Rank 0 - 20% complete. Time remaining: 1.94 mins
Rank 0 - 26% complete. Time remaining: 1.78 mins
Rank 0 - 33% complete. Time remaining: 1.58 mins
Rank 0 - 40% complete. Time remaining: 1.39 mins
Rank 0 - 46% complete. Time remaining: 1.24 mins
Rank 0 - 53% complete. Time remaining: 1.04 mins
Rank 0 - 60% complete. Time remaining: 54.02 sec
Rank 0 - 66% complete. Time remaining: 46.43 sec
Rank 0 - 73% complete. Time remaining: 39.42 sec
Rank 0 - 80% complete. Time remaining: 28.92 sec
Rank 0 - 87% complete. Time remaining: 19.25 sec
Rank 0 - 93% complete

...

Resuming computation. 0% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 - 6% complete. Time remaining: 2.59 mins
Rank 0 - 13% complete. Time remaining: 2.39 mins
Rank 0 - 20% complete. Time remaining: 2.08 mins
Rank 0 - 26% complete. Time remaining: 1.85 mins
Rank 0 - 33% complete. Time remaining: 1.67 mins
Rank 0 - 40% complete. Time remaining: 1.46 mins
Rank 0 - 46% complete. Time remaining: 1.22 mins
Rank 0 - 53% complete. Time remaining: 1.08 mins
Rank 0 - 60% complete. Time remaining: 56.61 sec
Rank 0 - 66% complete. Time remaining: 46.54 sec
Rank 0 - 73% complete. Time remaining: 37.59 sec
Rank 0 - 80% complete. Time remaining: 28.71 sec
Rank 0 - 87% complete. Time remaining: 18.99 sec
Rank 0 - 93% complete

/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Resuming computation. 0% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 - 6% complete. Time remaining: 3.76 mins
Rank 0 - 13% complete. Time remaining: 2.92 mins
Rank 0 - 20% complete. Time remaining: 2.46 mins
Rank 0 - 26% complete. Time remaining: 2.09 mins
Rank 0 - 33% complete. Time remaining: 1.87 mins
Rank 0 - 40% complete. Time remaining: 1.44 mins
Rank 0 - 46% complete. Time remaining: 1.27 mins
Rank 0 - 53% complete. Time remaining: 1.12 mins
Rank 0 - 60% complete. Time remaining: 59.0 sec
Rank 0 - 66% complete. Time remaining: 48.35 sec
Rank 0 - 73% complete. Time remaining: 40.31 sec
Rank 0 - 80% complete. Time remaining: 29.27 sec
Rank 0 - 87% complete. Time remaining: 18.96 sec
Rank 0 - 93% complete.

/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/alk224/miniconda3/envs/rapid_fitting/lib/python3.11/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Resuming computation. 0% completed already
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 - 6% complete. Time remaining: 2.32 mins
Rank 0 - 13% complete. Time remaining: 2.0 mins
Rank 0 - 20% complete. Time remaining: 1.8 mins
Rank 0 - 26% complete. Time remaining: 1.69 mins
Rank 0 - 33% complete. Time remaining: 1.74 mins
Rank 0 - 40% complete. Time remaining: 1.5 mins
Rank 0 - 46% complete. Time remaining: 1.33 mins
Rank 0 - 53% complete. Time remaining: 1.17 mins
Rank 0 - 60% complete. Time remaining: 58.49 sec
Rank 0 - 66% complete. Time remaining: 41.54 sec
Rank 0 - 73% complete. Time remaining: 34.14 sec
Rank 0 - 80% complete. Time remaining: 27.18 sec
Rank 0 - 87% complete. Time remaining: 18.26 sec
Rank 0 - 93% complete. T

### Checks the results to make sure it was saved correctly

In [8]:
# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Noisy_Data_2
    ├ Noisy_Data_3
    ├ Noisy_Data_4
    ├ Noisy_Data_5
    ├ Noisy_Data_6
    ├ Noisy_Data_7
    ├ Noisy_Data_8
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
├ Noisy_Data_1_SHO_Fit
  --------